In [1]:
import ast
import pandas as pd
import numpy as np
import csv
import json
import os
import pickle

## Importing basic patent data

In [2]:
ipcr = pd.read_csv('raw_data/ipcr.tsv', sep='\t')

/Users/quentinciccarone/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
ipcr.head()

,uuid,patent_id,classification_level,section,ipc_class,subclass,main_group,subgroup,symbol_position,classification_value,classification_status,classification_data_source,action_date,ipc_version_indicator,sequence
0,00005z3qh82fwpo5r1oupwpr3,6864832,NaN,G,01,S,013,/42,NaN,NaN,NaN,NaN,NaN,NaN,0
1,0000662nssr53hdo3lp92sz26,9954111,A,H,01,L,27,1156,L,I,B,H,2018-04-24,2017-01-01,8
2,00008u9j3g8oivqtuc1dqayb1,10048897,A,G,06,F,12,891,L,I,B,H,2018-08-14,2016-01-01,7
3,00008v5gnw215cdjozwehxqky,10694566,A,H,4,W,4,0,F,I,B,H,2020-06-23,2018-01-01,0
4,0000hj3ytmy8g9l2qa5x1hta5,D409748,NaN,D,24,04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [4]:
assignee = pd.read_csv('raw_data/rawassignee.tsv', sep='\t')

In [5]:
assignee.head()

,uuid,patent_id,assignee_id,rawlocation_id,type,name_first,name_last,organization,sequence
0,0000c0e5-81d5-4a02-bb9e-d5c3ef596900,4488683,org_zzDG6gSOdiYZdFsxQuQR,b44f6bf0-1f14-4b25-9ab6-06945ff1e8e1,3.0,NaN,NaN,Metal Works Ramat David,0
1,0000p94wkezw94s8cz7dbxlvz,5856666,org_fBtpUrdoVp5Lzvqma3Lv,orskbf54s58e97lkmw8na5rpx,2.0,NaN,NaN,U.S. Philips Corporation,0
2,00013vk881wap9u4mbo7lwwhp,5204210,org_uBBq49OpEQSGb2SJJBBT,mue862v5lcjdhzqqk86ei75kj,2.0,NaN,NaN,Xerox Corporation,0
3,000192sn2u10kzpikl4s7h3r0,5302149,org_ObwJtJBwY2t1dFv6oAAj,o1h9dqdv0yq7dt1b1vmrcal9h,3.0,NaN,NaN,Commonwealth Scientific & Industrial Research ...,1
4,0001ce9b-3c22-4f3a-b621-6c4e3ed85792,D397841,org_q4Ruupmeytq4y41ZiXdM,83c2755a-df62-4f4f-8509-43b9dcfeb038,3.0,NaN,NaN,"adidas, AG",0


## Canonical dataset

In [8]:
assignee_0 = assignee[['patent_id','assignee_id','organization']]

# list of organisation_id/true_name_organization
assignee_1 = assignee[['assignee_id','organization']]
assignee_1['organization'] = assignee_1['organization'].str.upper()

# list of  patent_id / organisation_id
assignee_2 = assignee[['patent_id','assignee_id']]


<ipython-input-8-8782019bf8df>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assignee_1['organization'] = assignee_1['organization'].str.upper()


In [9]:
ipcr_1 = ipcr[['patent_id','ipc_version_indicator']]

# conversion en date
ipcr_1['ipc_version_indicator']=pd.to_datetime(ipcr_1['ipc_version_indicator'],format='%Y-%m-%d', errors='ignore')

# filtrage pour enlever les patent qui n'ont pas de date
ipcr_1= ipcr_1[ipcr_1['ipc_version_indicator']>pd.datetime(1800,1,1)]

ipcr_1 = ipcr_1.set_index('patent_id')

# filtrage qui permet d'enlever les patents en double
ipcr_1 = ipcr_1[~ipcr_1.index.duplicated(keep='first')]


<ipython-input-9-8d40f726771a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipcr_1['ipc_version_indicator']=pd.to_datetime(ipcr_1['ipc_version_indicator'],format='%Y-%m-%d', errors='ignore')
<ipython-input-9-8d40f726771a>:7: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  ipcr_1= ipcr_1[ipcr_1['ipc_version_indicator']>pd.datetime(1800,1,1)]


In [17]:
# canonique_1 = assignee_1
# pickle.dump( canonique_1, open( "canonique_1.p", "wb" ) )

In [16]:
with open("canonique_1.p", "rb") as cano1:
    canonique_1 = pickle.load(cano1)
canonique_1.head()

,assignee_id,organization
0,org_zzDG6gSOdiYZdFsxQuQR,METAL WORKS RAMAT DAVID
1,org_fBtpUrdoVp5Lzvqma3Lv,U.S. PHILIPS CORPORATION
2,org_uBBq49OpEQSGb2SJJBBT,XEROX CORPORATION
3,org_ObwJtJBwY2t1dFv6oAAj,COMMONWEALTH SCIENTIFIC & INDUSTRIAL RESEARCH ...
4,org_q4Ruupmeytq4y41ZiXdM,"ADIDAS, AG"


In [15]:
# canonique_2 = pd.merge(ipcr_1,assignee_2, how='inner',on = 'patent_id')
# pickle.dump( canonique_2, open( "canonique_2.p", "wb" ) )

In [18]:
with open("canonique_2.p", "rb") as cano2:
    canonique_2 = pickle.load(cano2)
canonique_2.head()

,patent_id,ipc_version_indicator,assignee_id
0,9954111,2017-01-01,org_wYgU2Jhd2MTGAna87vpi
1,10048897,2016-01-01,org_ONzMjdbZXiKfw4L0cXl6
2,10694566,2018-01-01,org_dfvuIWENawcU6lTd1Z3w
3,7645556,2006-01-01,org_pCbqlmAg8wlWzoi18ITD
4,8524174,2006-01-01,org_7P7texQL0Q4WvpEEvDg8


In [19]:
canonique_2.groupby(by='assignee_id').count().sort_values(by=['patent_id'],ascending = False).head()

,patent_id,ipc_version_indicator
assignee_id,,
org_ONzMjdbZXiKfw4L0cXl6,97202,97202
org_pCbqlmAg8wlWzoi18ITD,68914,68914
org_eAKK85fawH0NS7AdXOig,44778,44778
org_g8U335TH48QmGJOIQnNl,32299,32299
org_OrmhECOcsM3rq5b7Pxfe,31143,31143


In [20]:
canonique_1[canonique_1['assignee_id']=='org_ONzMjdbZXiKfw4L0cXl6'].head()

,assignee_id,organization
81,org_ONzMjdbZXiKfw4L0cXl6,INTERNATIONAL BUSINESS MACHINES CORPORATION
124,org_ONzMjdbZXiKfw4L0cXl6,INTERNATIONAL BUSINESS MACHINES CORPORATION
127,org_ONzMjdbZXiKfw4L0cXl6,INTERNATIONAL BUSINESS MACHINES CORPORATION
166,org_ONzMjdbZXiKfw4L0cXl6,INTERNATIONAL BUSINESS MACHINES CORPORATION
171,org_ONzMjdbZXiKfw4L0cXl6,INTERNATIONAL BUSINESS MACHINES CORPORATION


### Importing M&A of startups data 

In [10]:
crunchbase_1 = pd.read_csv("/Volumes/Samsung_T5/patent_similarity_data/CRUNCHBASE_EXPORT/acquisitions_private_hardware_us.csv")

In [11]:
n = 0
firm_list = []
for k in range(100):
    acquiree = list(crunchbase_1['Acquiree Name'])[k]
    acquirer = list(crunchbase_1['Acquirer Name'])[k]

    has_aquiree_patent = 0
    has_aquirer_patent = 0
    
    has_aquiree_patent = sum(set(assignee_1['organization'].str.contains(acquiree.upper(), regex=True, na=False)))
    has_aquirer_patent = sum(set(assignee_1['organization'].str.contains(acquirer.upper(), regex=True, na=False)))

    value = min(has_aquiree_patent,has_aquirer_patent)
    n += value
    if value ==1:
        firm_list.append([acquiree.upper(),acquirer.upper()])
    

/Users/quentinciccarone/opt/anaconda3/lib/python3.8/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [12]:
n

31

In [13]:
firm_list

[['CGTECH', 'SANDVIK COROMANT'],
 ['KASTEN', 'VEEAM SOFTWARE'],
 ['BOX ROBOTICS', 'SEEGRID'],
 ['BOLT SOFTWARE', 'ECI SOFTWARE SOLUTIONS'],
 ['NEVION', 'SONY'],
 ['ARCHER SOFTWARE', 'CPRIME'],
 ['AUTOCRIB', 'SNAP-ON'],
 ['PACIFIC STAR COMMUNICATIONS', 'CURTISS-WRIGHT'],
 ['CLOUDISTICS', 'FUNGIBLE'],
 ['ZENIMAX', 'MICROSOFT'],
 ['CRADLEPOINT', 'ERICSSON'],
 ['PORTWORX', 'PURE STORAGE'],
 ['UNIVA', 'ALTAIR ENGINEERING'],
 ['TRACFONE WIRELESS', 'VERIZON COMMUNICATIONS'],
 ['ULC ROBOTICS', 'SPX CORPORATION'],
 ['HIGHFIVE', 'DIALPAD'],
 ['SPACES', 'APPLE'],
 ['PRECO', 'SENSATA TECHNOLOGIES'],
 ['STOCKWELL', '365 RETAIL MARKETS'],
 ['ALG', 'J.D. POWER AND ASSOCIATES'],
 ['PELCO', 'MOTOROLA SOLUTIONS'],
 ['PYPE', 'AUTODESK'],
 ['DISPLAYLINK', 'SYNAPTICS'],
 ['FLO TECHNOLOGIES', 'MOEN INCORPORATED'],
 ['SILVER PEAK', 'HEWLETT PACKARD ENTERPRISE'],
 ['ELECTRIC IMP', 'TWILIO'],
 ['NET IRRIGATE', 'LINDSAY CORPORATION'],
 ['DATRIUM', 'VMWARE'],
 ['MARBLE', 'CATERPILLAR'],
 ['ZOOX', 'AMAZON'],
 ['A

In [29]:
infile = open('/Volumes/Samsung_T5/patent_similarity_data/raw_data/most_sim.json','r',encoding='utf-8')

In [30]:
def similarity_patents_betw_2_firme(firms):
    acquiree = firms[0]
    acquirer = firms[1]
    
    assignee_id_acquiree = assignee_1[assignee_1['organization'].str.contains(acquiree, regex=True, na=False)]['assignee_id'].iloc[0]
    assignee_id_acquirer = assignee_1[assignee_1['organization'].str.contains(acquirer, regex=True, na=False)]['assignee_id'].iloc[0]

    patents_acquiree = set(assignee_2[assignee_2['assignee_id']==assignee_id_acquiree]["patent_id"])
    patents_acquirer = set(assignee_2[assignee_2['assignee_id']==assignee_id_acquirer]["patent_id"])

    
    similarity_acquiree_dico = {}
    similarity_acquirer_dico = {}

    for sim in infile:
        json_load = json.loads(sim)
        if json_load[0] in patents_acquiree:
            similarity_acquiree_dico[json_load[0]] = json_load[1]
        if json_load[0] in patents_acquirer:
            similarity_acquirer_dico[json_load[0]] = json_load[1] 
        
    return(similarity_acquiree_dico,similarity_acquirer_dico)
            

In [16]:
result_similarity = {}
for aree_arer in firm_list:
    sim_aree_arer = similarity_patents_betw_2_firme(aree_arer)
    patent_acquiree = set(sim_aree_arer[0].keys())
    patent_acquirer = set(sim_aree_arer[1].keys())

    for patent in patent_acquiree:
        print(patent)
        temp_dataframe = pd.DataFrame(sim_aree_arer[0][patent],columns=['patent_id','sim'])
        temp_dataframe = temp_dataframe[temp_dataframe['patent_id'].isin(patent_acquirer)]
        if len(temp_dataframe)>0:
            result_similarity[aree_arer[0]] = max(result_similarity.get(aree_arer[0],0),temp_dataframe['sim'].max())
        else:
            result_similarity[aree_arer[0]] = result_similarity.get(aree_arer[0],0)
        print(len(temp_dataframe))

10124544
0
9545759
0
6496382
0
5914862
0
5680459
0
6195858
0
6075707
0
7413040
0
10346151
0
10387167
0
9706198
0
10129539
0
9917798
0
9172501
0
10038651
1
9684572
0
9172507
0
7086558
0
9225102
0
10469268
0
7573713
0
8270325
0
7817589
0
7535861
0
9160619
0
8654749
0
10075413
0
10469867
1
10341678
0
7522165
2
7212202
1
9877531
1
8970585
0
10271055
3
6404426
4
8189000
2
10148978
0
10177957
1
10110417
1
9356835
0
9237102
0
8249052
0
7764784
1
9294353
0
10135677
0
8421637
0
9276940
1
7962569
0
10389583
1
8830075
0
8732808
1
9021081
0
8971209
0
8477639
0
9992062
0
9270571
0
9584406
1
8644272
0
10326652
1
9167437
0
8480761
0
9232461
1
9094280
0
9586117
0
10188925
0
6476166
0
7579415
0
7343225
0
8703884
0
6828394
0
8067518
0
9644053
0
9221937
0
6806328
0
6930070
0
7718566
0
8153068
0
7259125
0
8329835
0
7910668
0
8378043
0
6841631
0
8981021
0
7973112
0
6858689
0
7838605
0
6528597
0
8722804
0
7479529
0
6995109
0
6960634
0
8101691
0
6864206
0
8084560
0
8202940
0
8722820
0
7855253
0
8129483
0
775

3
6938864
0


In [18]:
# pickle.dump( result_similarity, open( "result_similarity_test.p", "wb" ) )

In [23]:
with open("result_similarity_test.p", "rb") as result_sim:
    result_similarity = pickle.load(result_sim)

In [24]:
result_similarity_final_useful = {}
for firm in result_similarity.keys():
    if result_similarity[firm]>0:
        result_similarity_final_useful[firm] = result_similarity[firm]

In [25]:
result_similarity_final_useful

{'NEVION': 0.33737462759017944,
 'ZENIMAX': 0.4011276662349701,
 'CRADLEPOINT': 0.3633490800857544,
 'SPACES': 0.41777321696281433,
 'PELCO': 0.3618326187133789,
 'DISPLAYLINK': 0.3821371793746948,
 'SILVER PEAK': 0.4321938157081604,
 'DATRIUM': 0.4869513511657715,
 'ZOOX': 0.4608118534088135}

In [83]:
len(result_similarity_final_useful) / 100

0.09

In [31]:
Dat_VMARE = similarity_patents_betw_2_firme(['DATRIUM', 'VMWARE'])

In [34]:
datrium_id = assignee_1[assignee_1['organization'].str.contains('DATRIUM', regex=True, na=False)]['assignee_id'].iloc[0]
datrium_id

'org_NsPj2a4iD80LdYxRz2kT'

In [40]:
patent_acquiree = Dat_VMARE[0].keys()

In [48]:
for patent in patent_acquiree:
    print(patent)
    temp_dataframe = pd.DataFrame(Dat_VMARE[0][str(patent)],columns=['patent_id','sim'])
    temp_filter = list(temp_dataframe['patent_id'].isin(patent_acquiree))

# to remove the patents from the acquiree company, not to false the results
    temp_filter = [not boolean for boolean in temp_filter]
    temp_dataframe = temp_dataframe[temp_filter]
    
# print the maximum similarity between the acquiree patent and the other patents which exist in the database
    print(temp_dataframe['sim'].max())

10061706
0.4604795575141907
10089013
0.5034605860710144
10146684
0.45062124729156494
10180948
0.4766446352005005
10235044
0.5269159078598022
10359945
0.46825987100601196
9417955
0.44092288613319397
9639268
0.4494040012359619
9703490
0.4483810067176819
